In [14]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [15]:
data = pd.read_csv('../data/data_w_pop.csv')
data = data.sort_values('pop2020')
# Problems with San Diego and Alexandria, count_qualified = 15 and 6
data = data[~data['region_name'].isin(['San Diego','Alexandria'])]

In [3]:
def estimate(json_file,flat_fraction):
    # Each row in dataset contains json-like data structure, converts
    sizes = json.loads(json_file)
    # Assumptions for parameters
    flat_fraction = flat_fraction
    kw_per_m2 = .15
    threshold = 929.0304
    # Initialize lists
    kws = []
    flat_roof_size = []
    roofs = []
    # Loop through each bin and the distribution of the capacity                                 
    for size in sizes:
        # Predicted rooftop solar capacity in KWs DC
        kw_install = size[0]
        # Number of roofs in this bin
        n_roof = size[1]
        # Convert to watts, divide by kw / m2, multiply by fraction flat 
        m2 = kw_install * flat_fraction / kw_per_m2
        # Append to lists
        kws.append(kw_install)        
        flat_roof_size.append(m2)
        roofs.append(n_roof)
    # Convert to numpy for element-wise multiplication property
    flat_roof_size = np.array(flat_roof_size) 
    roofs = np.array(roofs)
    kws = np.array(kws)
    # Only considers roofs above 10,000 Square Feet
    size_above = flat_roof_size[flat_roof_size > threshold]
    roofs_above = roofs[flat_roof_size > threshold] 
    kws_above = kws[flat_roof_size > threshold]    
    # return (total meter squared surface area, number of roofs / farms, total capacity loss to farming)
    return (np.sum(size_above * roofs_above),np.sum(roofs_above), np.sum(kws_above * roofs_above * flat_fraction))

In [4]:
n = data.shape[0]

p_factor = 21.5
frac_farmed = 0.6

p_by_city = []
m2_total_by_city = []
n_roofs_by_city = []
kwhs_loss_by_city = []

flat_fraction_by_city = data['number_of_panels_f'] / data['number_of_panels_total']
kwh_frac_total_by_city = data['yearly_sunlight_kwh_total'] / data['kw_total']

cities = data['install_size_kw_buckets_json'].to_numpy()

for idx, city in enumerate(cities):
    # Get capacity total for each city
    flat_fraction = flat_fraction_by_city.iloc[idx]
    kwh_fraction = kwh_frac_total_by_city.iloc[idx]
    m2_total, n_roofs_total, capacity_loss = estimate(city,flat_fraction)

    p = m2_total  * p_factor * frac_farmed
    kwh = capacity_loss * kwh_fraction

    
    p_by_city.append(p)
    m2_total_by_city.append(m2_total)
    n_roofs_by_city.append(n_roofs_total)
    kwhs_loss_by_city.append(kwh)

In [5]:
# Assign Variables
data['p'] = p_by_city
data['m2_total'] = m2_total_by_city
data['farms'] = n_roofs_by_city
data['frac_farms'] = data['farms'] / data['count_qualified']
data['p_per_capita'] = data['p'] / data['pop2020']
data['kwh_loss'] = kwhs_loss_by_city
data['kwh_loss_pct'] = data['kwh_loss'] / data['yearly_sunlight_kwh_total'] * 100
data[['region_name','state_name','pop2020','p','m2_total','farms','frac_farms','count_qualified','p_per_capita','kwh_loss','kwh_loss_pct','yearly_sunlight_kwh_total']].sort_values('count_qualified',ascending=True).iloc[:30]



,region_name,state_name,pop2020,p,m2_total,farms,frac_farms,count_qualified,p_per_capita,kwh_loss,kwh_loss_pct,yearly_sunlight_kwh_total
26,Arlington,Virginia,231803,1.079868e+06,8.371072e+04,31,0.072093,430.0,4.658560,1.522512e+07,54.948122,2.770817e+07
20,Denton,Texas,142173,5.787955e+04,4.486787e+03,2,0.000225,8907.0,0.407106,9.198591e+05,0.665136,1.382964e+08
112,North Las Vegas,Nevada,253923,1.086277e+06,8.420755e+04,41,0.002731,15011.0,4.277980,2.028939e+07,5.114404,3.967108e+08
125,Paterson,New Jersey,145871,2.876339e+06,2.229720e+05,120,0.006194,19375.0,19.718373,3.893896e+07,11.752998,3.313108e+08
89,McKinney,Texas,208487,6.473809e+05,5.018456e+04,32,0.001544,20730.0,3.105138,9.995426e+06,2.074980,4.817120e+08
195,Yonkers,New York,199021,3.717611e+06,2.881869e+05,147,0.006350,23150.0,18.679493,4.949132e+07,12.164440,4.068524e+08
191,Bridgeport,Connecticut,143010,3.591013e+06,2.783731e+05,142,0.006047,23481.0,25.110220,4.885696e+07,11.050044,4.421427e+08
86,Macon,Georgia,152519,5.973989e+06,4.630999e+05,228,0.008544,26686.0,39.168817,9.016867e+07,11.911042,7.570175e+08
106,Bellevue,Washington,154647,3.803604e+06,2.948530e+05,161,0.005798,27770.0,24.595392,4.225836e+07,7.771178,5.437832e+08
69,Jersey City,New Jersey,271099,1.554036e+07,1.204679e+06,341,0.011749,29024.0,57.323552,2.104095e+08,33.707385,6.242238e+08


In [6]:
data['p'].sum()

3564060918.9627

In [7]:
data[['region_name','state_name','pop2020','p','m2_total','farms','frac_farms','count_qualified','p_per_capita','kwh_loss','kwh_loss_pct','yearly_sunlight_kwh_total']].describe()

,pop2020,p,m2_total,farms,frac_farms,count_qualified,p_per_capita,kwh_loss,kwh_loss_pct,yearly_sunlight_kwh_total
count,1.950000e+02,1.950000e+02,1.950000e+02,195.000000,195.000000,195.000000,195.000000,1.950000e+02,195.000000,1.950000e+02
mean,4.140635e+05,1.827724e+07,1.416840e+06,594.989744,0.007315,85427.020513,47.773604,2.746966e+08,11.950284,2.315222e+09
std,7.157477e+05,2.386896e+07,1.850307e+06,749.633491,0.006574,92188.753309,37.378277,3.534164e+08,8.604361,2.456594e+09
min,1.420490e+05,5.787955e+04,4.486787e+03,2.000000,0.000183,430.000000,0.407106,9.198591e+05,0.156549,2.770817e+07
25%,1.799875e+05,4.844255e+06,3.755236e+05,192.000000,0.003726,40162.000000,20.480381,7.020038e+07,6.048634,1.125807e+09
50%,2.318030e+05,1.152090e+07,8.930927e+05,386.000000,0.006155,55577.000000,41.490932,1.680270e+08,10.480953,1.634277e+09
75%,3.894495e+05,2.067470e+07,1.602690e+06,650.000000,0.009220,81836.500000,65.739166,3.144103e+08,16.175236,2.419065e+09
max,8.323340e+06,2.077664e+08,1.610592e+07,6815.000000,0.072093,647621.000000,221.361256,3.076044e+09,54.948122,1.833701e+10
